In [ ]:
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import os
import time
import numpy as np
import glob
import matplotlib.pyplot as plt
import PIL
import imageio
from IPython import display
from imgaug import augmenters 
from numpy import loadtxt
from pylab import rcParams
from tensorflow.keras.layers import InputLayer, Conv2D, Flatten, Dense, Conv2DTranspose, Reshape
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib


# LOAD DATASETS

In [ ]:
def load_dataset(batch_size=1000):
    (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

    train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
    test_images = test_images.reshape(test_images.shape[0], 28, 28, 1).astype('float32')
    # Normalizing the images to the range of [0., 1.]
    train_images /= 255.
    test_images /= 255.

    TRAIN_BUF = 60000
    TEST_BUF = 10000

    BATCH_SIZE = batch_size
    # convert dataset to batches of data 
    train_dataset_image = tf.data.Dataset.from_tensor_slices(train_images).batch(BATCH_SIZE)
    train_dataset_label = tf.data.Dataset.from_tensor_slices(train_labels).batch(BATCH_SIZE)
    train_dataset = tf.data.Dataset.zip((train_dataset_image, train_dataset_label)).shuffle(TRAIN_BUF)

    test_dataset_image = tf.data.Dataset.from_tensor_slices(test_images).batch(BATCH_SIZE)
    test_dataset_label = tf.data.Dataset.from_tensor_slices(test_labels).batch(BATCH_SIZE)
    test_dataset = tf.data.Dataset.zip((test_dataset_image, test_dataset_label)).shuffle(TEST_BUF)

    return train_dataset, test_dataset

# PLOT GENERATED IMAGES AND LATENT SPACE DISTRIBUTION 

In [ ]:
# regenerate images from test set 
# 5 indicative images are reconstructed 
def plot_reconstructed_test_data(model, test_dataset):
    n = 5
    sample_dataset = test_dataset
    x_input, y_input = next(sample_dataset.__iter__())
    x_input_sample, y_input_sample = map(lambda x: x[:n], (x_input, y_input)) 
    z = model.encode(x_input_sample)[0].numpy()

    fig, axarr = plt.subplots(2, 5, figsize=(10, 5))
    x_input_sample = x_input_sample.numpy().reshape([n, 28, 28])
    x_output = model.decode(z, apply_sigmoid=True).numpy().reshape([n, 28, 28])

    for i in range(n):
        axarr[0, i].axis('off')
        axarr[1, i].axis('off')
        axarr[0, i].imshow(x_input_sample[i], cmap='binary')
        axarr[1, i].imshow(x_output[i], cmap='binary')

In [ ]:
# latent space distribution 
# lor latent dim 2 
def latent_space_distribution(model, test_dataset):
    n = 5
    sample_dataset = test_dataset
    x_input, y_input = next(sample_dataset.__iter__())
    x_input_sample, y_input_sample = map(lambda x: x[:n], (x_input, y_input))
    z, _ = model.encode(x_input)
    labels = y_input.numpy()
    z1, z2 = z.numpy().T[0], z.numpy().T[1]

    colors = matplotlib.cm.rainbow(np.linspace(0, 1, 10))
    cs = [colors[y] for y in labels]
    classes = list(range(10))

    recs = []
    for i in range(0, len(cs)):
        recs.append(mpatches.Rectangle((0, 0), 1, 1, fc=cs[i]))

    fig_dist = plt.figure(figsize=(8, 8))
    ax_dist = fig_dist.add_subplot(111)
    ax_dist.legend(recs, classes, loc=0)
    ax_dist.scatter(z1, z2, color=cs)


In [ ]:
# generate images giving to the decoder a latent vector 
def generated_images(model, test_dataset):
    n = 20
    f, axarr = plt.subplots(n, n, figsize=(12, 12))
    f.subplots_adjust(hspace=0., wspace=-0.)
    for i, z1 in enumerate(np.linspace(-2, 2, n)):
        for j, z2 in enumerate(np.linspace(-2, 2, n)):
            z = np.array([[z1, z2]])
            generated_img = model.decode(z, apply_sigmoid=True).numpy().reshape([28, 28])
            axarr[i, j].axis('off')
            axarr[i, j].imshow(generated_img, cmap='binary')





In [ ]:

def plot_VAE(model, test_dataset):
  plot_reconstructed_test_data(model, test_dataset)
  latent_space_distribution(model, test_dataset)
  generated_images(model, test_dataset)

  n = 16
  num_classes = 10
  f, axarr = plt.subplots(num_classes, n, figsize=(n, num_classes))
  f.subplots_adjust(hspace=0., wspace=-0.)
  for i in range(num_classes):
      for j, z_j in enumerate(np.linspace(-2, 2, n)):
          z = np.array([[z_j, 0]])
          z = tf.convert_to_tensor(z, dtype=tf.float32)
          generated_img = model.decode(z, apply_sigmoid=True).numpy().reshape([28, 28])
          axarr[i, j].axis('off')
          axarr[i, j].imshow(generated_img, cmap='binary')



# MODEL VAE 

In [ ]:

def encoder (latent_dim):
        encoder=tf.keras.Sequential([
            # input layer : image                                                       
            tf.keras.layers.InputLayer(input_shape=[28, 28, 1]),
            tf.keras.layers.Flatten(),
            # 2 dense layers 
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(64, activation='relu'),
            # output layer : for each latent variable we are
            # going to canculate its mean and variance                 
            tf.keras.layers.Dense(latent_dim * 2),    # [means, stds]
        ])
        return encoder

def decoder (latent_dim):
      decoder=tf.keras.Sequential([
            # input layer : a value for each latent variable  
            tf.keras.layers.InputLayer(input_shape=[latent_dim]),
            # 2 dense layers 
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(128, activation='relu'),
            # calculate and reconstruct the output to the shape shape as input image 
            tf.keras.layers.Dense(28 * 28 * 1),
            tf.keras.layers.Reshape(target_shape=[28, 28, 1]),
      ])
      return decoder

class VAE(tf.keras.Model):
    def __init__(self, latent_dim: int):
        super(VAE, self).__init__()
        self.latent_dim = latent_dim
        self.inference_net = encoder(self.latent_dim )
        self.generative_net = decoder(self.latent_dim)
  
    def encode(self, x):
        # call encoder
        mean_logvar = self.inference_net(x)
        N = mean_logvar.shape[0]
        # First outpout of encoder: the mean of the distribution q(zi|x) 
        mean = tf.slice(mean_logvar, [0, 0], [N, self.latent_dim])
        # Second outpout of encoder: the covariance table calculated for the distribution q(zi|x)  
        logvar = tf.slice(mean_logvar, [0, self.latent_dim], [N, self.latent_dim])
        return mean, logvar

    def decode(self, z, apply_sigmoid=False):
        logits = self.generative_net(z)
        if apply_sigmoid:
            probs = tf.sigmoid(logits)
            return probs
        else:            
            return logits

    # function used for reparametrization trick ( mean + conv * eps) 
    def reparameterize(self, mean, logvar):
        eps = tf.random.normal(shape=mean.shape)
        return eps * tf.exp(logvar * .5) + mean





# TRAIN 

In [ ]:
# For training our algorithm we have to maximize the ELBO 
# Instead of that convert this maximization problem to a minimization problem 
# by calculating the -ELBO

class VAETrain:
    @staticmethod
    def compute_loss(model, x):
        mean, logvar = model.encode(x)
        z = model.reparameterize(mean, logvar)
        x_logits = model.decode(z)

        # cross_ent = - marginal likelihood
        cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logits, labels=x)
        marginal_likelihood = - tf.reduce_sum(cross_ent, axis=[1, 2, 3])
        marginal_likelihood = tf.reduce_mean(marginal_likelihood)

        # Our KL divergence loss can be rewritten in the formula defined above (Wiseodd, 2016).
        # https://wiseodd.github.io/techblog/2016/12/10/variational-autoencoder/
        KL_divergence = tf.reduce_sum(mean ** 2 + tf.exp(logvar) - logvar - 1, axis=1)
        KL_divergence = tf.reduce_mean(KL_divergence)

        ELBO = marginal_likelihood - KL_divergence
        # Adam optimizer used finds the min of a function 
        # for this reason we convert ELBO (mazimization problem)
        # to -ELBO 
        # now using as a loos function = -ELBO we convert the initial maximization project 
        # to a minimization problem
        loss = -ELBO
        return loss

    @staticmethod
    def compute_gradients(model, x, optimizer):
        with tf.GradientTape() as tape:
            loss = VAETrain.compute_loss(model, x)
        loss_init = loss
        gradients= tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        return loss_init




# TRAIN

In [ ]:
def train_VAE(latent_dim=2, epochs=100, lr=1e-4, batch_size=1000):
    model = VAE(latent_dim)
    train_dataset, test_dataset = load_dataset(batch_size=batch_size)

    optimizer = tf.keras.optimizers.Adam(lr)

    # for each epoch train all the batches 
    for epoch in range(1, epochs + 1):
        t = time.time()
        last_loss = 0
        for train_x, _ in train_dataset:
            loss = VAETrain.compute_gradients(model, train_x, optimizer)
            last_loss = loss
        if epoch % 50 == 0:
            print('Epoch {}, Loss: {}, Remaining Time at This Epoch: {:.2f}'.format(
                epoch, last_loss, time.time() - t
            ))

    plot_VAE(model, test_dataset)

    return model





# Simulations

In [ ]:
train_VAE(latent_dim=2,epochs=150)

In [ ]:
train_VAE(latent_dim=2,epochs=100)